<a href="https://colab.research.google.com/github/barronh/pseudonetcdf_examples/blob/main/camx_smat_input/camx_smat_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare SMAT Inputs from CAMX

    author: Barron H. Henderson
    date: 2020-08-24
    last updated: 2021-11-15
    PseudoNetCDF version: latest

# Overview

This example shows how make SMAT inputs from CAMx output (avrg) files. The inputs are hourly gridded. They are converted to maximum daily 8-hour averages and then subset for 3x3 arrays around each monitor. The results are put in the SMAT-CE model input format.

# Install  Libraries

* Use `apt-get` to install `libgeos-dev` for projection and transformations.
* Use `pip` to install `basemap`  for mapping.
* Use `pip` to install `PseudoNetCDF` for many file format support.
* Can take a couple minutes

In [1]:
!pip install -qq pyproj pseudonetcdf

     |████████████████████████████████| 6.3 MB 4.7 MB/s 
     |████████████████████████████████| 298 kB 31.1 MB/s 


# Import Libraries

* `PseudoNetCDF` for camx file reading
* `pandas` for csv writing
* `numpy` for numeric processing
* `os` for path manipulation
* `urlretrieve` to download obs


In [2]:
import PseudoNetCDF as pnc
import pandas as pd
import numpy as np
import os
from urllib.request import urlretrieve

# Downloading the CAMx v7 Test Case

* Go to http://camx.com and "Download," then "CAMx Test Case
* Right click on one of the "Test problem outputs" links, and choose "Copy link address"
* Update the camxtarurl below.


In [3]:
# Requires input from the user
# most recently, the url was:
# 'https://camx-wp.azurewebsites.net/getmedia/CAMx7-10-test_run-outputs-GFortran_compiler-210105.tgz'
camxtarurl = input('Enter CAMx test case url: ')

Enter CAMx test case url: https://camx-wp.azurewebsites.net/getmedia/CAMx7-10-test_run-outputs-GFortran_compiler-210105.tgz


* If not downloaded, download it.
* If not extracted, extract two model output days as netcdf.

In [4]:
camxtarpath = os.path.basename(camxtarurl)
print(camxtarpath)
if not os.path.exists(camxtarpath):
    urlretrieve(camxtarurl, filename=camxtarpath)

CAMx7-10-test_run-outputs-GFortran_compiler-210105.tgz


In [5]:
!tar xzf {camxtarpath} --wildcards "*.avrg.grd02.nc"
camxoutpaths = !ls ./outputs/*.avrg.grd02.nc
camxoutpaths

['./outputs/CAMx.v7.10.36.12.20160610.avrg.grd02.nc',
 './outputs/CAMx.v7.10.36.12.20160611.avrg.grd02.nc']

# Download Observations

* Used for monitor locations
* If not donwloaded, download.
* Load latitude and longitude

In [6]:
obspath = '8hour_44201_2016.zip'
downloadroot = 'https://aqs.epa.gov/aqsweb/airdata/'
if not os.path.exists(obspath):
  urlretrieve(downloadroot + obspath, obspath)

In [7]:
def keyclean(k):
    k = k.replace(' ', '')
    k = k.replace('1st', 'First')
    k = k.replace('Latitude', 'LAT')
    k = k.replace('Longitude', 'LONG')
    k = k.replace('DateLocal', 'Date')
    return k

obsdf = pd.read_csv(obspath, usecols=['Latitude', 'Longitude']).rename(columns=keyclean)

# Maximum Daily Average 8-hour

* Define a mda8 function
* Currently not providing time zone support. -- Just an example.

In [8]:
def mda8(x):
  """
  Calculate maximum daily 8-hour average.

  
  Arguments
  ---------
  x : array-like
    vector of time values
  
  Returns
  -------
    Daily maximum of the 8-hour average
  
  Notes
  -----
  No tz support yet
  """
  return np.convolve(np.ones(8)/8., x, mode='full')[7:].reshape(-1, 24)[:, :17].max(1)

# Process CAMx Files

In [9]:
camxf = pnc.pncmfopen(
    camxoutpaths, format='ioapi', stackdim='TSTEP'
).subset(['O3']).slice(
    ROW=slice(1, -1), # Nested inputs have invalid zeros in the edge
    COL=slice(1, -1)  # This removes the edges
)
camxmda8f = camxf.apply(TSTEP=mda8)

# Extending data to be a quarter
camxmda8f = camxmda8f.slice(TSTEP=np.arange(2, dtype='i').repeat(45, 0))
camxmda8f.updatetflag(startdate=camxf.getTimes()[0], overwrite=True, tstep=240000)

o3mda8 = camxmda8f.variables['O3']

**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  Got duplicate variables for X without stackable dimension; first value retained
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  Got duplicate variables for Y without stackable dimension; first value retained
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  New time is unstructured


In [10]:
times = pd.to_datetime([t.replace(tzinfo=None) for t in camxmda8f.getTimes()])
I, J = np.meshgrid(np.arange(camxmda8f.NCOLS), np.arange(camxmda8f.NCOLS))
lon, lat = camxmda8f.ij2ll(I, J)
ID = I * 1000 + J + 1001

**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF


In [11]:
lldf = obsdf.groupby(['LONG', 'LAT'], as_index=False).first()
iidx, jidx = camxf.ll2ij(
    lldf.LONG.values, lldf.LAT.values, clean='mask'
)
# increased buffer edge by 1
# this allows a 3x3 array inside
# the domain
valididx = (
    (iidx >= 1) & (iidx < (camxf.NCOLS - 1)) &
    (jidx >= 1) & (jidx < (camxf.NROWS - 1))                                  
)

**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF


In [12]:
smatobsdf = lldf[valididx].copy()
smatobsdf['K'] = jidx[valididx] * 0
smatobsdf['J'] = jidx[valididx]
smatobsdf['I'] = iidx[valididx]
gdf = smatobsdf.groupby(['K', 'J', 'I'], as_index=False).max()

In [13]:
dataframes = []
times = camxmda8f.getTimes()
timesstr = pd.to_datetime(times[:, None].repeat(gdf.shape[0], 1).ravel()).strftime('%Y%m%d')
myk = gdf.K.values

# Used for matrix construction
naway = 1
dcells = range(-naway, naway + 1)

for i in dcells:
  myi = gdf['I'].values + i
  for j in dcells:
    myj = gdf['J'].values + j
    o3 = o3mda8[:, myk, myj, myi] * 1000
    data = dict(
        _ID=((myi + 1) * 1000 + (myj + 1))[None, :].repeat(times.size, 0).ravel(),
        LONG=lon[myj, myi][None, :].repeat(times.size, 0).ravel(),
        LAT=lat[myj, myi][None, :].repeat(times.size, 0).ravel(),
        DATE=timesstr,
        O3=o3.ravel()
    )
    tmpdf = pd.DataFrame(data=data)
    dataframes.append(tmpdf)

outdf = pd.concat(dataframes)
outdf['_TYPE'] = ''


In [14]:
def savesmat(df, outpath):
    # get rid of overlaps
    df = df.groupby(['DATE', '_ID'], as_index=False).first()
    keys = ['_ID', '_TYPE', 'LAT', 'LONG', 'DATE', 'O3']
    outf = open(outpath, 'w')
    outf.write('Day\n')
    df.loc[:, keys].sort_values(['DATE', '_ID']).to_csv(outf, index=False, float_format='%.6f')
    outf.close()


In [15]:
savesmat(outdf, 'CAMX_FOR_SMAT.csv')

In [16]:
testdf = outdf.copy()
testdf['O3'] *= 0.9
savesmat(testdf, 'CAMX_90pct_FOR_SMAT.csv')

# Ran with SMAT

All sites had a 90% RRF as expected.
